In [16]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression 
from sklearn import preprocessing, feature_extraction
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer


In [3]:
data=pd.read_csv('C:\\Users\\Emmanuel\\Documents\\movies.csv')
data.head()

,presupuesto,genero,lenguaje,popularidad,productores,pais,ventas,duracion,titulo,puntuacion,n_votos
0,NaN,Comedy,en,8.387519,Sandollar Productions,United States of America,76578911.0,106.0,Father of the Bride Part II,5.7,173.0
1,NaN,Drama,en,0.894647,Miramax,South Africa,676525.0,106.0,"Cry, the Beloved Country",6.7,13.0
2,3500000.0,Comedy,en,14.569650,New Line Cinema,United States of America,28215918.0,91.0,Friday,7.0,513.0
3,NaN,Comedy,en,8.963037,Paramount Pictures,United States of America,32.0,87.0,Black Sheep,6.0,124.0
4,12000000.0,Comedy,en,9.592265,Universal Pictures,United States of America,41205099.0,92.0,Happy Gilmore,6.5,767.0


In [4]:
from sklearn.base import TransformerMixin

In [27]:
class ColumnExtractor(TransformerMixin):

    def __init__(self, columns):
        self.columns = columns
        
    def transform(self, X, **transform_params):
        return X[self.columns].to_numpy()
        
    def fit(self, X, y=None, **fit_params):
        return self

In [34]:
col_numericas=['presupuesto', 'popularidad','duracion', 'puntuacion', 'n_votos']
imputador =  SimpleImputer(missing_values=np.nan, strategy='mean')
escalador = preprocessing.StandardScaler()
transformador_numerico = Pipeline(
     [('imputador', imputador), ('escalador', escalador)]
)

pipeline_numerico = Pipeline([
    ['selector_numerico', ColumnExtractor(columns=col_numericas)],
    ['transformador_numerico', transformador_numerico]
])

data_Procesada=pipeline_numerico.fit_transform(data)

In [30]:
data_Procesada.shape


(1344, 6)

In [35]:
estimador = LinearRegression()
pipeline_estimador = Pipeline([
    ('procesador', pipeline_numerico),
    ('estimador', estimador)
])

In [37]:
from sklearn.model_selection import cross_val_score
data_con_ventas=data[data['ventas'].notnull()]

resultados = cross_val_score(pipeline_estimador,
                            X=data_con_ventas,
                            y=data_con_ventas['ventas'],
                            scoring="neg_mean_absolute_error")

In [38]:
resultados


array([-31479522.91195995, -22551325.95234929, -20864221.66966091,
       -25286141.57398211, -18844821.79016568])

In [39]:
resultados.mean()

-23805206.77962359